# Beer Scraper

6 Notebooks like this were uploaded to AWS EC2 instances in order to scrape the desired data and deposite said data in a MongoDB also hosted on AWS.

As this is more-or-less a demo notebook or model of the actual, none of the code has been executed.

I would also like to note that I ran this scraper at the beginning of 2018 so website structure could have changed since then make this code no longer work.

In [ ]:
# Docker container I was using did not come with pymongo preinstalled.
!pip install pymongo

In [ ]:
### Necessary Imports
import pandas as pd

import re
import bs4
from bs4 import BeautifulSoup
import requests
import urllib

import numpy as np

import time

from IPython.display import clear_output

In [ ]:
# Connecting to MongoDatabase
from pymongo import MongoClient
# Actual IP and Ports changed for security reasons.
client = MongoClient('250.250.250.250',20000)

# setting path to collection
Beers = client['Beers']
beer_coll = Beers['Beer_final']

In [ ]:
# Gathering all the beerstyles for a reference dictionary

beer_styles_page = requests.get('https://www.beeradvocate.com/beer/style/')

beer_styles_soup = BeautifulSoup(beer_styles_page.content,"html5lib")

beer_styles = {}
for beer in beer_styles_soup.find_all(name='tr'):
    for style in beer.find_all('a'):
        name = style.text
        link = 'https://www.beeradvocate.com'+style['href']
        beer_styles[name] = link 

### Division by Style

Once I ran the above and collected all the different styles and generated their associated landing pages, I randomly divided them up into 6 groups.  Each of the 6 scrapers got one of those groups. This example code has `scraper_group` as the variable containing an example set of beer styles to get.

In [ ]:
# Quality Reviews Extractor Function Used on individual Beers.
def quality_review_extractor(reviews_url, reviews_dict):
    
    # request page contents and convert to soup object
    QR_beer_r = requests.get(reviews_url)
    QR_beer_r_soup = BeautifulSoup(QR_beer_r.content, 'html5lib')
    
    # find all individual reviews
    for ind_review in QR_beer_r_soup.find_all('div', attrs = {'id':'rating_fullview_container'}): 
        full_ind_review = ind_review.text # contains username and unwated numeric reviews

        # unwanted attributes are contained in <span class="muted">"Unwated text</span>
        numeric_review = ind_review.find_all(name ='span',attrs={'class':'muted'})
    
        # cleans out unwanted attributes (username, datetime, numeric ratings)
        for reviewer in numeric_review:
            unwanted_attr = reviewer.text
            # replaces unwanted aspects
            full_ind_review = full_ind_review.replace(unwanted_attr, '')
            # scrubs out the 'Total Score' attribute
            clean_review = re.sub( '^(.*)(%)',"",string = full_ind_review)
        
        # if the length of the remaining review is greater than 25 keep
        if len(clean_review) > 25:
            reviews_dict[str(len(reviews_dict)+1)] = clean_review

In [ ]:
# Table row scraper
def beer_table_scrape(table_row, current_type):
    beer_tag = table_row.find_all('td')
    # Try to get the general beer info from the row
    try:
        beer_name = beer_tag[0].text
        brewery_name = beer_tag[1].text
        abv = beer_tag[2].text
        ratings = beer_tag[3].text

        avg_score = beer_tag[4].text
        beer_page = 'https://www.beeradvocate.com'+(beer_tag[0].find('a')['href']+'?sort=topr&start=')
    except:
        pass
            
        
    quality_reviews = {} # empty dict to append reviews to
    reviews_page = 0 
    
    
    while len(quality_reviews) < 25 and reviews_page < 1000: # while we have less than 25 good reviews.  
        reviews_page_url = beer_page+str(reviews_page) #url formula to change pages
        
 
        
        print("Current Beer : ",beer_name,
              " --- Pages Scraped : ",int(reviews_page/25),
              " --- Quality Reviews : ",len(quality_reviews))
        
        print(reviews_page_url)
        clear_output(wait=True) # clear print output so I dont stack print statements.  
        
        quality_review_extractor(reviews_page_url, quality_reviews) # hit the function
        
        quality_reviews['Beer_Name'] = beer_name
        quality_reviews['Brewery_Name'] = brewery_name
        quality_reviews['ABV'] = abv
        quality_reviews['Type'] = current_type
        
        
        reviews_page += 25 # value increment to change to next 
        
        pause = np.random.lognormal(mean=1.5, sigma=0.4, size=1) #Lognormal dist, avg a 4.5sec pause.
        
        time.sleep(pause) #take a nap so beeradvocate does not get suspicious.  
    
    beer_coll.insert_one(quality_reviews)

In [ ]:
scraper_group = ['American Brown Ale', 'Belgian Pale Ale', 'Czech Pilsener', 'English Brown Ale',
                 'Milk / Sweet Stout', 'Winter Warmer', 'English Stout', 'Flanders Oud Bruin', 
                 'Low Alcohol Beer', 'Gueuze', 'Belgian IPA', 'Irish Dry Stout', 'Kellerbier / Zwickelbier', 
                 'Foreign / Export Stout', 'Rauchbier', 'Faro', 'Bock', 'Belgian Strong Dark Ale']

In [ ]:
#for each beer in sublist
for individual_style in scraper_group: #random style list
    base_beer_style_url = beer_styles[individual_style]+'?sort=revsD&start=' # base type url
    page_count_value = 0
    break_light = False
    
    #while we can still get access
    while requests.get(base_beer_style_url+str(page_count_value)).status_code == 200 and break_light == False:
        
        # use individual style as our key to get the appropriate url & soup that shit
        beer_style_table = BeautifulSoup(requests.get(base_beer_style_url+str(page_count_value)).content, 'html5lib')

        # Loops through each row in the table on a page
        for table_row in beer_style_table.find(name ='table', attrs = {'width':'100%'}).find_all('tr')[3:53]:
            
            if int(table_row.find_all('td')[3].text.replace(',','')) < 100:
                break_light = True
                break

                
            # perform that action, if it returns false, because we met the criteria
            beer_table_scrape(table_row, individual_style)  #if the table scrape ends because of reviews being too low
            
                 # increment the page count
        page_count_value += 50